<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO COMPLETE WORKING LAUNCHER
# Fixed all dependency issues + Persistent Google Drive storage
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
from pathlib import Path
import shutil

print("🚀 MY-AI-GIZMO COMPLETE INSTALLER")
print("="*70)

# --- STEP 1: Mount Google Drive ---
print("\n📂 Step 1: Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("   ✅ Mounted")

# --- STEP 2: Setup Directories ---
print("\n📁 Step 2: Setting up directories...")

DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"

for d in [DRIVE_BASE, MODELS_DIR, USER_DATA_DIR]:
    d.mkdir(exist_ok=True)

print(f"   ✅ Base: {DRIVE_BASE}")

# --- STEP 3: Clone Repository ---
if not (REPO_DIR / "server.py").exists():
    print("\n📥 Step 3: Cloning repository...")
    if REPO_DIR.exists():
        shutil.rmtree(REPO_DIR)
    os.chdir(DRIVE_BASE)
    subprocess.run(["git", "clone", "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"], check=True)
    print("   ✅ Cloned")
else:
    print("\n✅ Step 3: Repository exists")

os.chdir(REPO_DIR)

# --- STEP 4: Link User Data ---
print("\n🔗 Step 4: Linking user data to Drive...")

local_user_data = REPO_DIR / "user_data"
if local_user_data.exists() and not local_user_data.is_symlink():
    for item in local_user_data.rglob("*"):
        if item.is_file():
            rel = item.relative_to(local_user_data)
            dest = USER_DATA_DIR / rel
            dest.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(item, dest)
    shutil.rmtree(local_user_data)

if not local_user_data.exists():
    local_user_data.symlink_to(USER_DATA_DIR)
    print("   ✅ Linked")

(USER_DATA_DIR / "logs").mkdir(exist_ok=True)

# --- STEP 5: Environment Variables ---
print("\n🔧 Step 5: Environment setup...")
os.environ['MPLBACKEND'] = 'Agg'
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
os.environ['CMAKE_ARGS'] = '-DLLAMA_CUBLAS=off'  # Force CPU build
print("   ✅ Environment configured")

# --- STEP 6: Install Core Dependencies ---
print("\n📦 Step 6: Installing dependencies...")
print("   This may take 2-3 minutes on first run...")

# Upgrade pip
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip"],
               check=False, stdout=subprocess.DEVNULL)

# Install essential packages first
print("   Installing core packages...")
essential = [
    "wheel",
    "setuptools",
    "numpy",
    "requests",
    "tqdm",
    "pyyaml"
]

for pkg in essential:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                  check=False, stdout=subprocess.DEVNULL)

# Install llama-cpp-python for CPU
print("   Installing llama-cpp-python (CPU)...")
subprocess.run([
    sys.executable, "-m", "pip", "install",
    "llama-cpp-python", "--no-cache-dir",
    "--force-reinstall",
    "-q"
], check=False, env=dict(os.environ, CMAKE_ARGS='-DLLAMA_CUBLAS=off'))

# Install llama_cpp_binaries (the missing module)
print("   Installing llama_cpp_binaries...")
subprocess.run([
    sys.executable, "-m", "pip", "install", "-q",
    "llama-cpp-binaries"
], check=False)

# Try requirements files
installed_from_req = False
for req in ["requirements_cpu_only.txt", "requirements.txt"]:
    if Path(req).exists():
        print(f"   Installing from {req}...")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install",
            "-r", req, "-q"
        ], capture_output=True, text=True)

        if result.returncode == 0:
            print(f"   ✅ Installed from {req}")
            installed_from_req = True
            break

# Install remaining common packages
print("   Installing additional packages...")
additional = [
    "torch",
    "transformers",
    "gradio>=3.50.0",
    "accelerate",
    "markdown",
    "Pillow",
    "safetensors",
    "sentencepiece",
    "protobuf"
]

for pkg in additional:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                  check=False, stdout=subprocess.DEVNULL)

print("   ✅ All dependencies installed")

# --- STEP 7: Fix UI Theme ---
print("\n🔧 Step 7: Patching UI...")

ui_file = Path("modules/ui.py")
if ui_file.exists():
    with open(ui_file, 'r', encoding='utf-8') as f:
        content = f.read()

    if 'button_shadow_hover' in content and 'if False:' not in content:
        content = content.replace(
            'if not shared.args.old_colors:',
            'if False:  # Disabled for Colab'
        )
        with open(ui_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print("   ✅ UI patched")
    else:
        print("   ✅ UI compatible")
else:
    print("   ⚠️  UI file not found")

# --- STEP 8: Download Model ---
print("\n⬇️  Step 8: Checking model...")

MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

if MODEL_FILE.exists():
    size_gb = MODEL_FILE.stat().st_size / (1024**3)
    print(f"   ✅ Model exists ({size_gb:.2f} GB)")
else:
    print("   📥 Downloading model (3.8 GB - ONE TIME ONLY)...")
    print("   This will take 3-5 minutes...")
    url = "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"

    result = subprocess.run([
        "wget", "-q", "--show-progress",
        "-O", str(MODEL_FILE), url
    ], check=True)

    if MODEL_FILE.exists():
        print(f"   ✅ Downloaded ({MODEL_FILE.stat().st_size / (1024**3):.2f} GB)")
    else:
        print("   ❌ Download failed!")
        sys.exit(1)

# --- STEP 9: Link Models Directory ---
print("\n🔗 Step 9: Linking models...")

repo_models = REPO_DIR / "models"
if not repo_models.is_symlink():
    if repo_models.exists():
        shutil.rmtree(repo_models)
    repo_models.symlink_to(MODELS_DIR)
    print("   ✅ Models linked")
else:
    print("   ✅ Already linked")

# --- STEP 10: Verify Installation ---
print("\n🔍 Step 10: Verifying installation...")

checks = {
    "server.py": REPO_DIR / "server.py",
    "Model file": MODEL_FILE,
    "User data": USER_DATA_DIR,
    "Modules dir": REPO_DIR / "modules"
}

all_good = True
for name, path in checks.items():
    if path.exists():
        print(f"   ✅ {name}")
    else:
        print(f"   ❌ {name} missing!")
        all_good = False

if not all_good:
    print("\n❌ Installation incomplete!")
    sys.exit(1)

# --- STEP 11: Interactive Configuration ---
print("\n" + "="*70)
print("⚙️  CONFIGURATION")
print("="*70)

# Hardware
print("\n🖥️  Hardware Mode:")
print("1. CPU only (Recommended for Colab)")
print("2. GPU (Requires GPU runtime)")
print("3. Auto-detect")

while True:
    hw = input("\nChoice (1-3): ").strip()
    if hw in ['1', '2', '3']:
        break
    print("❌ Enter 1, 2, or 3")

if hw == '1':
    use_cpu = True
    hw_name = "CPU"
elif hw == '2':
    use_cpu = False
    hw_name = "GPU"
else:
    try:
        has_gpu = subprocess.run(['nvidia-smi'], capture_output=True).returncode == 0
        use_cpu = not has_gpu
        hw_name = "GPU (detected)" if has_gpu else "CPU (no GPU)"
    except:
        use_cpu = True
        hw_name = "CPU (fallback)"

print(f"✅ Using: {hw_name}")

# Options
print("\n🔧 Additional Options:")
print("1. Enable API")
print("2. Verbose logging")
print("3. Both API + Verbose")
print("4. None (minimal)")

while True:
    opts = input("\nChoice (1-4): ").strip()
    if opts in ['1', '2', '3', '4']:
        break
    print("❌ Enter 1, 2, 3, or 4")

enable_api = opts in ['1', '3']
enable_verbose = opts in ['2', '3']

# --- STEP 12: Build Launch Command ---
print("\n" + "="*70)
print("🚀 LAUNCHING MY-AI-GIZMO")
print("="*70)

cmd = [sys.executable, "server.py"]

# Hardware flags
if use_cpu:
    cmd.extend(["--cpu", "--threads", "4"])

# Core flags
cmd.extend([
    "--listen",
    "--share",
    "--model", str(MODEL_FILE),
    "--loader", "llama.cpp"  # Force llama.cpp loader
])

# Optional flags
if enable_api:
    cmd.extend(["--api", "--public-api"])
if enable_verbose:
    cmd.append("--verbose")

# Display config
print(f"\n📝 Launch Configuration:")
print(f"   Hardware: {hw_name}")
print(f"   Model: {MODEL_FILE.name}")
print(f"   Loader: llama.cpp")
print(f"   API: {'Enabled' if enable_api else 'Disabled'}")
print(f"   Verbose: {'Yes' if enable_verbose else 'No'}")
print(f"   Chats save to: {USER_DATA_DIR / 'logs'}")

print("\n" + "="*70)
print("⏳ Starting server...")
print("🔗 WATCH FOR GRADIO LINK BELOW ⬇️")
print("="*70 + "\n")

# --- STEP 13: Launch Server ---
try:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    found_link = False
    error_lines = []

    for line in process.stdout:
        print(line, end='')

        # Capture errors
        if 'error' in line.lower() or 'traceback' in line.lower():
            error_lines.append(line)

        # Highlight Gradio link
        if ('gradio.live' in line.lower() or
            ('running on' in line.lower() and 'http' in line)):
            if not found_link:
                print("\n" + "🎉"*35)
                print("✅ SERVER RUNNING! CLICK THE LINK ABOVE ☝️")
                print("🎉"*35)
                print("\n💡 USAGE TIPS:")
                print("   • Select mode in the web UI (Chat/Default/Notebook)")
                print("   • Conversations auto-save to Google Drive")
                print("   • Use 'Session' tab to manage saved chats")
                print("   • Press Ctrl+C here to stop server")
                print("="*70 + "\n")
                found_link = True

    return_code = process.wait()

    if return_code != 0 and not found_link:
        print("\n❌ Server exited with error")
        if error_lines:
            print("\n🔍 Error details:")
            for line in error_lines[-10:]:
                print(line, end='')

except KeyboardInterrupt:
    print("\n\n⏹️  Server stopped by user")
    print(f"💾 All chats saved in: {USER_DATA_DIR / 'logs'}")

except Exception as e:
    print(f"\n\n❌ ERROR: {e}")
    print("\n🔧 Troubleshooting:")
    print(f"   1. Check if model exists: {MODEL_FILE.exists()}")
    print(f"   2. Check working directory: {os.getcwd()}")
    print(f"   3. Try running manually:")
    print(f"      cd {REPO_DIR}")
    print(f"      python server.py --cpu --model {MODEL_FILE}")

    # Show command that failed
    print(f"\n📋 Command that was run:")
    print(f"   {' '.join(str(x) for x in cmd)}")

    # Try to get more error details
    print("\n🔍 Attempting to run with error capture...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.stderr:
        print("STDERR:")
        print(result.stderr[:1000])